# Scenario B: The Obstacle Course

Goal: Go from origin to $(2.5, 2.5)$.
Constraint: Pass through midpoint $(2.6, 1)$.
Variable Start: $y$ position is flexible near origin. We will try $y=0$ first.

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

import numpy as np
import matplotlib.pyplot as plt
from surfaces import ScenarioB
from physics import PhysicsEngine
from solver import ShootingSolver
from utils import plot_trajectory

In [2]:
surface = ScenarioB()
physics = PhysicsEngine(surface)
solver = ShootingSolver(physics)

midpoint = np.array([2.6, 1.0])
endpoint = np.array([2.5, 2.5])

def objective(params):
    vx0, vy0, T1, T2 = params
    if T1 <= 0 or T2 <= T1:
        return [100, 100, 100, 100]
        
    initial_state = [0, 0, vx0, vy0]
    
    # Integrate to T2, capturing T1
    sol = solver.integrate(initial_state, [0, T2])
    
    # Interpolate at T1
    y_at_T1 = sol.sol(T1)
    x1, y1 = y_at_T1[0], y_at_T1[1]
    
    # State at T2
    x2, y2 = sol.y[0, -1], sol.y[1, -1]
    
    return [x1 - midpoint[0], y1 - midpoint[1], x2 - endpoint[0], y2 - endpoint[1]]

# Initial guess [vx0, vy0, T1, T2]
# Need a reasonable guess. Maybe shoot towards midpoint first.
guess = [3.0, 1.0, 1.0, 2.0]

# Note: solve_ivp result object 'sol' has a .sol attribute if dense_output=True is used.
# We need to modify solver.integrate to use dense_output=True or just evaluate at T1 if it's in t_eval.
# Let's update the objective to use dense_output.

def objective_dense(params):
    vx0, vy0, T1, T2 = params
    if T1 <= 0.1 or T2 <= T1 + 0.1:
        return [100, 100, 100, 100]
        
    initial_state = [0, 0, vx0, vy0]
    
    # We need dense output to interpolate at T1
    from scipy.integrate import solve_ivp
    sol = solve_ivp(
        physics.equations_of_motion,
        [0, T2],
        initial_state,
        rtol=1e-8, atol=1e-8,
        dense_output=True
    )
    
    if not sol.success:
        return [100, 100, 100, 100]

    state_T1 = sol.sol(T1)
    state_T2 = sol.sol(T2)
    
    return [
        state_T1[0] - midpoint[0],
        state_T1[1] - midpoint[1],
        state_T2[0] - endpoint[0],
        state_T2[1] - endpoint[1]
    ]

sol_root = solver.solve(objective_dense, guess)

print("Solution:", sol_root.x)
print("Success:", sol_root.success)
print("Message:", sol_root.message)

KeyboardInterrupt: 

In [ ]:
if sol_root.success:
    vx0, vy0, T1, T2 = sol_root.x
    initial_state = [0, 0, vx0, vy0]
    sol_path = solver.integrate(initial_state, [0, T2])
    plot_trajectory(surface, sol_path, title="Scenario B Solution")
    
    # Verify points
    from scipy.integrate import solve_ivp
    sol_dense = solve_ivp(
        physics.equations_of_motion,
        [0, T2],
        initial_state,
        rtol=1e-8, atol=1e-8,
        dense_output=True
    )
    p1 = sol_dense.sol(T1)
    p2 = sol_dense.sol(T2)
    print(f"Point at T1: ({p1[0]:.4f}, {p1[1]:.4f}) vs Target ({midpoint[0]}, {midpoint[1]})")
    print(f"Point at T2: ({p2[0]:.4f}, {p2[1]:.4f}) vs Target ({endpoint[0]}, {endpoint[1]})")